In [18]:
!pip install python-dotenv --quiet


In [52]:
# -------------------------------------------------------------
# This script sets up the Colab environment for interacting with BigQuery and Google Drive.
#
# Purpose:
# - Authenticate the Colab session with Google Cloud services.
# - Initialize the BigQuery client for querying and managing datasets.
# - Mount Google Drive to access project files (e.g., saved CSVs, templates).
# - Load environment variables (such as API keys) securely from a .env file.
#
# Notes:
# - Must run this block at the start of every Colab session.
# - Assumes the .env file is stored at /MyDrive/google/.env.
# - Enables seamless access to BigQuery and external APIs (e.g., Google Places API).
# -------------------------------------------------------------

# %load /content/drive/MyDrive/ColabTemplates/colab_bigquery_startup.txt

# -------------------------------------------------------------
# Authenticate Colab session, initialize BigQuery client, mount Drive.
# -------------------------------------------------------------

# 1. Authenticate to Google Cloud
from google.colab import auth
auth.authenticate_user()
print("✅ Colab session authenticated.")

# 2. Initialize BigQuery client
from google.cloud import bigquery
client = bigquery.Client()

# 3. Print authenticated service account
from google.auth import default
creds, _ = default()
try:
    print(f"🔐 Authenticated as: {creds.service_account_email}")
except AttributeError:
    print("🔐 Authenticated, but could not retrieve email.")

# 4. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print("✅ Google Drive mounted.")


🔐 Authenticated as: default
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
# -------------------------------------------------------------
# This script generates a CSV containing unique restaurants from CleanedInspectionRow.
#
# Purpose:
# - Extract a deduplicated list of restaurants based on dba_name, address, zip, and state.
# - Apply a facility_type filter to include only true restaurants.
# - Save the resulting list as a CSV file in Google Drive for future batch enrichment.
#
# Notes:
# - Run this script only if a fresh unique restaurant list is needed.
# - Output is saved to /MyDrive/msds434_project/unique_restaurants_from_cleanedinspection.csv.
# - This file serves as the master source for Google Places enrichment batches.
# -------------------------------------------------------------


from google.cloud import bigquery
import pandas as pd
import os

# Initialize BigQuery client
client = bigquery.Client(project="hygiene-prediction-434")

# Query: get unique restaurants
query = """
SELECT DISTINCT
  dba_name,
  address,
  zip,
  state
FROM `hygiene-prediction-434.HygienePredictionRow.CleanedInspectionRow`
WHERE dba_name IS NOT NULL
  AND address IS NOT NULL
  AND zip IS NOT NULL
  AND state IS NOT NULL
  AND LOWER(facility_type) = 'restaurant'
ORDER BY dba_name, address
"""

# Run query and load to DataFrame
df_unique_restaurants = client.query(query).to_dataframe()

print(f"✅ Retrieved {len(df_unique_restaurants)} unique restaurants.")

# Save to Google Drive
# Set the correct path
folder_path = "/content/drive/MyDrive/msds434_project"

# Make sure the folder exists
os.makedirs(folder_path, exist_ok=True)

# Define the output file path
output_path = os.path.join(folder_path, "unique_restaurants_from_cleanedinspection.csv")

# Save the DataFrame
df_unique_restaurants.to_csv(output_path, index=False)

print(f"✅ Saved unique restaurant list to {output_path}")

✅ Retrieved 9552 unique restaurants.
✅ Saved unique restaurant list to /content/drive/MyDrive/msds434_project/unique_restaurants_from_cleanedinspection.csv


In [57]:
# -------------------------------------------------------------
# This script creates a new BigQuery dataset called 'RestaurantModeling'
# inside the 'hygiene-prediction-434' project.
#
# Purpose:
# - Organize all modeling-ready tables (RestaurantProfile, InspectionEvents).
# - Separate clean, structured data from raw inspection input.
# - Provide a foundation for feature engineering and model development.
#
# Notes:
# - The dataset location is set to 'US' to match existing tables.
# - Running this script is idempotent (safe to run multiple times).
# -------------------------------------------------------------


from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client(project="hygiene-prediction-434")

# Define the new dataset ID
dataset_id = "hygiene-prediction-434.RestaurantModeling"

# Configure the dataset
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"  # Match your other tables (important)

# Create the dataset
dataset = client.create_dataset(dataset, exists_ok=True)

print(f"✅ Dataset created or already exists: {dataset.dataset_id}")


✅ Dataset created or already exists: RestaurantModeling


In [103]:
# -------------------------------------------------------------
# This script creates the 'RestaurantProfile' table in the 'RestaurantModeling' dataset.
#
# Purpose:
# - Define the schema for the RestaurantProfile table, which stores enriched restaurant metadata.
# - Establish a structured, modeling-ready table based on Google Places enrichment.
# - Prepare for stable joins to inspections using place_id as the foreign key.
#
# Notes:
# - The 'place_id' is marked as REQUIRED since it is the primary key for joining.
# - Other fields such as rating, price_level, and categories are NULLABLE for flexibility.
# - The 'types' field is an ARRAY<STRING> to store multiple Google Places types.
# - The table will be created if it does not exist, or validated if it already does.
# - Only needs to run once
# -------------------------------------------------------------

# -------------------------------------------------------------
# Create the 'RestaurantProfile' table in the 'RestaurantModeling' dataset.
# Updated for parsed address and zip information.
# -------------------------------------------------------------

from google.cloud import bigquery

# Initialize BigQuery client
client = bigquery.Client(project="hygiene-prediction-434")

# Define full table ID
table_id = "hygiene-prediction-434.RestaurantModeling.RestaurantProfile"

# Define the updated schema
schema = [
    bigquery.SchemaField("place_id", "STRING", mode="REQUIRED"),         # Primary join key
    bigquery.SchemaField("dba_name", "STRING", mode="NULLABLE"),          # Original business name from inspection data
    bigquery.SchemaField("matched_name", "STRING", mode="NULLABLE"),      # Google Places matched business name
    bigquery.SchemaField("address", "STRING", mode="NULLABLE"),           # Street address parsed from Google Places
    bigquery.SchemaField("zip", "INTEGER", mode="NULLABLE"),              # ZIP code parsed from Google Places
    bigquery.SchemaField("rating", "FLOAT", mode="NULLABLE"),             # Google Places rating (1-5)
    bigquery.SchemaField("price_level", "INTEGER", mode="NULLABLE"),      # Google Places price level (0-4)
    bigquery.SchemaField("user_ratings_total", "INTEGER", mode="NULLABLE"), # Number of Google reviews
    bigquery.SchemaField("business_status", "STRING", mode="NULLABLE"),   # Business operational status (Operational, Permanently Closed, etc.)
    bigquery.SchemaField("types", "STRING", mode="REPEATED"),             # List of business types (restaurant, cafe, etc.)
    bigquery.SchemaField("category", "STRING", mode="NULLABLE"),          # Our engineered operational category (full_service, fast_food, etc.)
    bigquery.SchemaField("price_category", "STRING", mode="NULLABLE"),    # Our engineered price category (low, medium, high)
    bigquery.SchemaField("popularity_category", "STRING", mode="NULLABLE"),# Our engineered popularity category
]

# Create or recreate the table
table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table, exists_ok=True)

print(f"✅ Table created or updated: {table.project}.{table.dataset_id}.{table.table_id}")



✅ Table created or updated: hygiene-prediction-434.RestaurantModeling.RestaurantProfile


In [99]:
# -------------------------------------------------------------
# This script creates the 'InspectionEvents' table in the 'RestaurantModeling' dataset.
#
# Purpose:
# - Define the schema for InspectionEvents, which stores one row per inspection event.
# - Link each inspection to a restaurant via place_id (foreign key to RestaurantProfile).
# - Structure inspection metadata for easy ML feature engineering and prediction modeling.
#
# Notes:
# - 'inspection_id' is REQUIRED as a unique identifier.
# - 'place_id' is REQUIRED to link inspections to restaurant profiles.
# - 'violation_codes' is an ARRAY<STRING> to store multiple codes per inspection.
# - 'has_critical_violation' and 'num_violations' are engineered fields for ML use.
# - The table is created if it does not exist, or validated if it already does.
# -------------------------------------------------------------

from google.cloud import bigquery

# Initialize BigQuery client
client = bigquery.Client(project="hygiene-prediction-434")

# Define full table ID
table_id = "hygiene-prediction-434.RestaurantModeling.InspectionEvents"

# Define schema
schema = [
    bigquery.SchemaField("inspection_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("place_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("inspection_date", "DATE", mode="NULLABLE"),
    bigquery.SchemaField("inspection_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("result", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("violation_codes", "STRING", mode="REPEATED"),
    bigquery.SchemaField("num_violations", "INT64", mode="NULLABLE"),
    bigquery.SchemaField("has_critical_violation", "BOOL", mode="NULLABLE"),
    bigquery.SchemaField("risk", "STRING", mode="NULLABLE"),
]

# Create the table
table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table, exists_ok=True)

print(f"✅ Table created or updated: {table.project}.{table.dataset_id}.{table.table_id}")



✅ Table created or updated: hygiene-prediction-434.RestaurantModeling.InspectionEvents


In [92]:
# -------------------------------------------------------------
# Generate Critical Violation Codes Dynamically Based on Inspection History
#
# Purpose:
# - Select violation codes that occur frequently and are highly associated with inspection failures.
# - Create an adaptive, data-driven list of critical codes for has_critical_violation labeling.
#
# Notes:
# - Must be run before migrating or updating InspectionEvents table.
# - Saves CRITICAL_CODES to Drive for reproducibility if needed.
# -------------------------------------------------------------


# Query BigQuery to create df_violations first
query = """
WITH exploded AS (
  SELECT
    inspection_id,
    risk,
    results AS result,
    LOWER(risk) LIKE '%high%' AS is_high_risk,
    LOWER(results) = 'fail' AS is_fail,
    code
  FROM `hygiene-prediction-434.HygienePredictionRow.CleanedInspectionRow`,
  UNNEST(violation_codes) AS code
  WHERE code IS NOT NULL
)

SELECT
  code,
  COUNT(*) AS code_count,
  SUM(CAST(is_high_risk AS INT64)) AS high_risk_count,
  SUM(CAST(is_fail AS INT64)) AS fail_count,
  ROUND(SAFE_DIVIDE(SUM(CAST(is_high_risk AS INT64)), COUNT(*)), 3) AS pct_high_risk,
  ROUND(SAFE_DIVIDE(SUM(CAST(is_fail AS INT64)), COUNT(*)), 3) AS pct_fail
FROM exploded
GROUP BY code
ORDER BY code_count DESC
"""

df_violations = client.query(query).to_dataframe()
print(f"✅ Retrieved {len(df_violations)} rows of violation code analysis.")


# Set thresholds (adjust if needed)
MIN_CODE_COUNT = 100
MIN_PCT_FAIL = 0.7

# Filter violation codes dynamically
df_critical = df_violations[
    (df_violations["code_count"] >= MIN_CODE_COUNT) &
    (df_violations["pct_fail"] >= MIN_PCT_FAIL)
]

# Create the critical codes set
CRITICAL_CODES = set(df_critical["code"].astype(str))

print(f"✅ Selected {len(CRITICAL_CODES)} critical codes:")
print(sorted(CRITICAL_CODES))

# (Optional) Save to Drive
import json

critical_codes_path = "/content/drive/MyDrive/msds434_project/critical_codes.json"
with open(critical_codes_path, "w") as f:
    json.dump(sorted(list(CRITICAL_CODES)), f)

print(f"✅ Critical codes list saved to {critical_codes_path}")


✅ Retrieved 99 rows of violation code analysis.
✅ Selected 32 critical codes:
['10', '11', '12', '13', '14', '16', '17', '18', '19', '20', '21', '22', '23', '25', '26', '27', '29', '30', '35', '40', '42', '47', '49', '50', '53', '55', '57', '60', '75', '8', '80', '9']
✅ Critical codes list saved to /content/drive/MyDrive/msds434_project/critical_codes.json


In [96]:
# -------------------------------------------------------------
# Migrate inspection data into RestaurantModeling.InspectionEvents
# -------------------------------------------------------------

from google.cloud import bigquery
import pandas as pd
import json
import os

# Initialize BigQuery client
client = bigquery.Client(project="hygiene-prediction-434")

# Step 1: Query the existing inspection data
query = """
SELECT
  inspection_id,
  dba_name,
  address,
  zip,
  state,
  inspection_date,
  inspection_type,
  results AS result,
  violations,
  risk
FROM `hygiene-prediction-434.HygienePredictionRow.CleanedInspectionRow`
WHERE inspection_id IS NOT NULL
"""

df_inspections = client.query(query).to_dataframe()
print(f"✅ Retrieved {len(df_inspections)} rows from CleanedInspectionRow")

# Step 2: Transform fields

# Extract violation codes
def extract_violation_codes(violations):
    if pd.isna(violations):
        return []
    codes = []
    for part in violations.split("|"):
        part = part.strip()
        if part and part[0].isdigit():
            code = part.split(".")[0]
            codes.append(code)
    return codes

df_inspections["violation_codes"] = df_inspections["violations"].apply(extract_violation_codes)

# Count number of violations
df_inspections["num_violations"] = df_inspections["violation_codes"].apply(len)

# Load dynamic critical codes from JSON
critical_codes_path = "/content/drive/MyDrive/msds434_project/critical_codes.json"
if os.path.exists(critical_codes_path):
    with open(critical_codes_path, "r") as f:
        CRITICAL_CODES = set(json.load(f))
    print(f"✅ Loaded {len(CRITICAL_CODES)} dynamic critical codes.")
else:
    raise FileNotFoundError(f"❌ critical_codes.json not found at {critical_codes_path}")

# Compute has_critical_violation dynamically
def has_critical(codes):
    return any(code in CRITICAL_CODES for code in codes)

df_inspections["has_critical_violation"] = df_inspections["violation_codes"].apply(has_critical)

# Add placeholder place_id (to be filled in later)
df_inspections["place_id"] = None

# Step 3: Select columns in correct order
columns_for_insert = [
    "inspection_id", "place_id", "inspection_date",
    "inspection_type", "result", "violation_codes",
    "num_violations", "has_critical_violation", "risk"
]

df_ready = df_inspections[columns_for_insert]
print(f"✅ Prepared {len(df_ready)} rows for insertion into InspectionEvents")



✅ Retrieved 54236 rows from CleanedInspectionRow
✅ Loaded 32 dynamic critical codes.
✅ Prepared 54236 rows for insertion into InspectionEvents


In [100]:
# Insert prepared inspection data into BigQuery
table_id = "hygiene-prediction-434.RestaurantModeling.InspectionEvents"

# Ensure inspection_id is a string
df_ready = df_ready.copy()
df_ready["inspection_id"] = df_ready["inspection_id"].astype(str)


# Now insert
job = client.load_table_from_dataframe(
    df_ready,
    "hygiene-prediction-434.RestaurantModeling.InspectionEvents",
    job_config=bigquery.LoadJobConfig(write_disposition="WRITE_APPEND")
)

job.result()

print(f"✅ Successfully inserted {len(df_ready)} rows into InspectionEvents")


✅ Successfully inserted 54236 rows into InspectionEvents


In [101]:
query = """
SELECT
  COUNT(*) AS total_inspections,
  SUM(CASE WHEN place_id IS NULL THEN 1 ELSE 0 END) AS inspections_missing_place_id,
  SUM(CASE WHEN has_critical_violation THEN 1 ELSE 0 END) AS inspections_with_critical_violation
FROM `hygiene-prediction-434.RestaurantModeling.InspectionEvents`
"""

df_summary = client.query(query).to_dataframe()
df_summary


,total_inspections,inspections_missing_place_id,inspections_with_critical_violation
0,54236,54236,39396
